In [1]:
import os
os.system('python -m pip install --no-index --find-links=/kaggle/input/hnswlib/ hnswlib')

Looking in links: /kaggle/input/hnswlib/
Processing /kaggle/input/hnswlib/hnswlib-0.6.2-cp37-cp37m-linux_x86_64.whl


0

In [2]:
import numpy as np
import pandas as pd
import time
import math
from sklearn.metrics import f1_score
from torch.optim import Adam, SGD, AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, DataCollatorWithPadding
from transformers import BertTokenizer,AutoModel,AdamW,AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torch.nn.functional as F
from tqdm import tqdm
import copy
import torch.nn as nn
import os
import json
import gc
import random
from torch.cuda.amp import autocast, GradScaler
import hnswlib  # 在kaggle上这个库要提前离线安装一下

In [3]:
class CFG:
    input_path = 'LECR'
    model_path = '/kaggle/input/mdeberta-v3-base/mdeberta-v3-base' 
    scheduler = 'cosine'  # ['linear', 'cosine']
    batch_scheduler = True
    num_cycles = 0.5  # 1.5
    num_warmup_steps = 0
    max_input_length = 124
    epochs = 5  # 5
    encoder_lr = 20e-6
    decoder_lr = 1e-3
    min_lr = 0.5e-6
    eps = 1e-6
    betas = (0.9, 0.999)
    weight_decay = 0
    num_fold = 5
    batch_size = 32
    seed = 42
    OUTPUT_DIR = 'LECR'
    num_workers = 2
    device='cuda'
    print_freq = 100
    apex=False
    start_awp_epoch = 2 # 开始AWP epoch
    adv_lr = 1e-5 # AWP学习率
    adv_eps = 1e-3 # AWP epsilon
    adv_step = 1 # AWP step

In [4]:
class Custom_Bert_Simple(nn.Module):
    def __init__(self):
        super().__init__()

        self.base = AutoModel.from_pretrained(CFG.model_path)
        self.config = AutoConfig.from_pretrained(CFG.model_path)

    def forward(self,
        input_ids,
        attention_mask=None):
        output = self.base(input_ids=input_ids,attention_mask=attention_mask)
        output = output.last_hidden_state
        output = torch.mean(output, dim=1)
        return output

In [5]:
PTH_PATH = '/kaggle/input/lecr-baseline-mdeberta-v3-base-best0/LECRmicrosoft_mdeberta-v3-base_best0.pth'

In [6]:
model = Custom_Bert_Simple()
model.load_state_dict(torch.load(PTH_PATH),strict=False)
model.to(CFG.device)
model.eval()

Some weights of the model checkpoint at /kaggle/input/mdeberta-v3-base/mdeberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifer.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifer.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Custom_Bert_Simple(
  (base): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0): DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
              (dropo

In [7]:
DATA_PATH = '/kaggle/input/learning-equality-curriculum-recommendations/'

In [8]:
content_df = pd.read_csv(DATA_PATH + 'content.csv')
correlations_df = pd.read_csv(DATA_PATH + 'correlations.csv')
topics_df = pd.read_csv(DATA_PATH + 'topics.csv')
#topics_df = topics_df[topics_df['category']!='source'].reset_index(drop=True)
sub_df = pd.read_csv(DATA_PATH + 'sample_submission.csv')

In [9]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
class Testdataset(Dataset):
    def __init__(self,df,tokenizer):
        self.title = df['title'].values
        self.description = df['description'].values
        self.text = None
        if 'text' in df.columns:
            self.text = df['text'].values
        
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
    def __len__(self):
        return len(self.title)
    def __getitem__(self, item):
        
        input_text = self.title[item]
        if isinstance(input_text, float):
            input_text = ''
        if not isinstance(self.description[item], float):
            #print(self.description[item])
            input_text += ' ' + self.sep_token + ' ' + self.description[item]
        
        if self.text is not None and not isinstance(self.text[item], float):
            input_text += ' ' + self.sep_token + self.text[item]
            
        output = self.tokenizer(input_text, truncation=True, max_length=CFG.max_input_length, padding='max_length')
        
        return torch.as_tensor(output['input_ids'], dtype=torch.long), \
            torch.as_tensor(output['attention_mask'], dtype=torch.long)

In [11]:
topic_dataset = Testdataset(topics_df[topics_df['id'].isin(sub_df['topic_id'])], tokenizer)
content_dataset = Testdataset(content_df, tokenizer)
topic_loader = DataLoader(topic_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
content_loader = DataLoader(content_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

In [12]:
def infer(model, dataloader):
    res = []
    for batch in tqdm(dataloader):
        input_ids, attention_mask = [i.to(CFG.device) for i in batch]
        with torch.no_grad():
            output = model(input_ids, attention_mask)
            res.append(output.cpu().numpy())
    
    return np.vstack(res)

In [ ]:
topic_result = infer(model, topic_loader)
content_result = infer(model, content_loader)

In [ ]:
content_ids = [i for i in range(len(content_df))]

In [ ]:
def build_index(embeddings, ids):

    index = hnswlib.Index(space="cosine", dim=embeddings.shape[-1])

    # Initializing index
    # max_elements - the maximum number of elements (capacity). Will throw an exception if exceeded
    # during insertion of an element.
    # The capacity can be increased by saving/loading the index, see below.
    #
    # ef_construction - controls index search speed/build speed tradeoff
    #
    # M - is tightly connected with internal dimensionality of the data. Strongly affects memory consumption (~M)
    # Higher M leads to higher accuracy/run_time at fixed ef/efConstruction
    index.init_index(max_elements=embeddings.shape[0], ef_construction=200, M=160)

    # Controlling the recall by setting ef:
    # higher ef leads to better accuracy, but slower search
    index.set_ef(50)

    # Set number of threads used during batch search/construction
    # By default using all available cores
    index.set_num_threads(16)

    
    index.add_items(embeddings, ids)


    return index

In [ ]:
content_index = build_index(content_result, content_ids)
results = content_index.knn_query(topic_result, k = 5, num_threads = -1)

In [ ]:
pred = []
conten_uid = content_df['id']
for result in tqdm(results[0]):
    top_same = ' '.join(conten_uid[result].to_list())
    pred.append(top_same)

In [ ]:
sub_df['content_ids'] = pred
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv', index=None)